## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
notebook_input_path = '/kaggle/input/126-googleq-a-train-1fold-bert-base-unc-quest-ans/'
model_question_path_list = glob.glob(notebook_input_path + '*question.h5')
model_answer_path_list = glob.glob(notebook_input_path + '*answer.h5')
model_question_path_list.sort()
model_answer_path_list.sort()
print('Models to predict:', model_question_path_list + model_answer_path_list)

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict: ['/kaggle/input/126-googleq-a-train-1fold-bert-base-unc-quest-ans/model_fold_1_question.h5', '/kaggle/input/126-googleq-a-train-1fold-bert-base-unc-quest-ans/model_fold_1_answer.h5']
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
text_features_question = ['question_title', 'question_body']
text_features_answer = ['question_title', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
N_CLASS_QUESTION = len(question_target_cols)
N_CLASS_ANSWER = len(answer_target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test_question = compute_input_arays_2(test, text_features_question, tokenizer, MAX_SEQUENCE_LENGTH, ignore_first_sep=False)
X_test_answer = compute_input_arays_2(test, text_features_answer, tokenizer, MAX_SEQUENCE_LENGTH, ignore_first_sep=False)

# Model

In [7]:
def question_model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS_QUESTION, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

def answer_model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS_ANSWER, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for index, model_path in enumerate(model_question_path_list):
    model_question = question_model_fn()
    model_answer = answer_model_fn()
    model_question.load_weights(model_path)
    model_answer.load_weights(model_answer_path_list[index])
    
    question_preds = model_question.predict(X_test_question)
    answer_preds = model_answer.predict(X_test_answer)
    Y_test += np.hstack([question_preds, answer_preds]) / len(model_question_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.954565,0.647815,0.201075,0.621467,0.496942,0.372557,0.675942,0.593287,0.729911,...,0.924405,0.922444,0.576538,0.970264,0.953566,0.817849,0.019945,0.050943,0.858880,0.941216
1,46,0.854435,0.553192,0.007483,0.747594,0.736435,0.916896,0.507182,0.417338,0.144778,...,0.626741,0.950708,0.683519,0.969089,0.987007,0.881883,0.949633,0.110052,0.041605,0.908170
2,70,0.892738,0.650516,0.019259,0.714707,0.850326,0.905290,0.606310,0.418570,0.201012,...,0.846918,0.972891,0.607713,0.982886,0.980776,0.885397,0.029603,0.034252,0.977519,0.902335
3,132,0.845241,0.386692,0.006880,0.703912,0.752131,0.886394,0.527341,0.409782,0.186227,...,0.692258,0.949005,0.684287,0.971497,0.984787,0.907549,0.857401,0.171434,0.751348,0.883629
4,200,0.898313,0.451908,0.024001,0.790537,0.776139,0.864084,0.611418,0.623123,0.094821,...,0.637418,0.883826,0.642115,0.959163,0.962953,0.802051,0.266250,0.166524,0.588768,0.887558


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.878287,0.584844,0.032677,0.711368,0.783279,0.816730,0.552115,0.460556,0.262502,...,0.801503,0.934252,0.651184,0.964618,0.972027,0.859476,0.544441,0.143794,0.524345,0.910533
std,2812.670060,0.052627,0.133907,0.055400,0.087511,0.112004,0.147003,0.060584,0.089468,0.204890,...,0.087187,0.037152,0.057480,0.019645,0.019507,0.057873,0.351557,0.075432,0.335724,0.033715
min,39.000000,0.704443,0.351975,0.003303,0.310960,0.267137,0.210733,0.417573,0.328126,0.017828,...,0.581342,0.729714,0.390833,0.847334,0.849365,0.596626,0.000564,0.002921,0.008861,0.752639
25%,2572.000000,0.841722,0.472304,0.007153,0.664314,0.755996,0.792248,0.506353,0.397192,0.102400,...,0.733886,0.916590,0.617401,0.956915,0.964738,0.830229,0.158512,0.088028,0.191878,0.891928
50%,5093.000000,0.880731,0.564961,0.011607,0.719491,0.795283,0.865084,0.540484,0.430656,0.201074,...,0.803760,0.943565,0.652849,0.968914,0.978244,0.871367,0.654873,0.142747,0.519270,0.916601
75%,7482.000000,0.919069,0.690028,0.027217,0.760770,0.842874,0.907336,0.593554,0.504271,0.381482,...,0.880914,0.961248,0.686375,0.978197,0.985365,0.900173,0.884656,0.191630,0.868055,0.935047
max,9640.000000,0.979491,0.895993,0.395224,0.948792,0.976072,0.967909,0.734317,0.794689,0.889119,...,0.949286,0.989447,0.797659,0.992915,0.994103,0.957263,0.975583,0.385076,0.998359,0.971142
